In [ ]:
import warnings
import time
import copy
import random

import numpy as np
import pandas as pd

import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
train_X = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/train_x.npy", allow_pickle=True)
train_Y = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/train_y.npy", allow_pickle=True)
valid_X = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/valid_x.npy", allow_pickle=True)
valid_Y = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/valid_y.npy", allow_pickle=True)
test_X = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/test_x.npy", allow_pickle=True)
test_Y = np.load("C:/Users/goodc/OneDrive/Desktop/ComputerVision_DeepLearning/dataset/test_y.npy", allow_pickle=True)

print(train_X.shape, type(train_X))
print(train_Y.shape, type(train_X))
print(valid_X.shape, type(valid_X))
print(valid_Y.shape, type(valid_X))
print(test_X.shape, type(test_X))
print(test_Y.shape, type(test_X))

In [ ]:
def image_normalization(arr):
    return (arr - arr.min())/(arr.max()-arr.min())

def disable_ax_ticks(ax):
    ax.set_yticks([], minor=True)

def show_example(x, y):
    fig = plt.figure()
    main_ax = fig.add_subplot()
    fig.suptitle('label = '+ str(y))
    main_ax.imshow(image_normalization(np.moveaxis(x, 0, -1)))
    disable_ax_ticks(main_ax)
    plt.show()

# def show_example_with_white_crosshair(x, y):
#     fig = plt.figure()
#     main_ax = fig.add_subplot()
    
#     # Find the center point of the image
#     center_x = x.shape[2] // 2
#     center_y = x.shape[1] // 2

#     # Convert image data from (3, 64, 64) to (64, 64, 3) for displaying purposes
#     img_display = image_normalization(np.moveaxis(x, 0, -1))
    
#     # Draw a white crosshair at the center of the image
#     img_display[center_y, :, :] = [1, 1, 1]  # Horizontal line
#     img_display[:, center_x, :] = [1, 1, 1]  # Vertical line

#     fig.suptitle('label = '+ str(y))
    
#     # Display the image
#     main_ax.imshow(img_display)
#     disable_ax_ticks(main_ax)
#     plt.show()

count = train_X.shape[0]
number = random.randrange(count)
show_example(train_X[number], train_Y[number])

In [ ]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if not train_on_gpu:
    print("Using GPU is not available.", "Device:", device)
else:
    print("Using GPU is available.", "Device:", device)

In [ ]:
def Model_Vgg19():
    # ========================================================================================================================

    num_classes = 4
    batch_size = 128
    num_epochs = 60

    # ========================================================================================================================

    for i in range(len(train_X)):
        for j in range(len(train_X[i])):
            min = train_X[i][j].min()
            max = train_X[i][j].max()
            train_X[i][j] = abs((train_X[i][j] - min) / (max - min))

    for i in range(len(valid_X)):
        for j in range(len(valid_X[i])):
            min = valid_X[i][j].min()
            max = valid_X[i][j].max()
            valid_X[i][j] = abs((valid_X[i][j] - min) / (max - min))

    for i in range(len(test_X)):
        for j in range(len(test_X[i])):
            min = test_X[i][j].min()
            max = test_X[i][j].max()
            test_X[i][j] = abs((test_X[i][j] - min) / (max - min))

    # ========================================================================================================================

    train_X_toTensor = torch.Tensor(train_X)
    train_Y_toTensor = torch.Tensor(train_Y)
    train_Y_toLongTensor = train_Y_toTensor.type(torch.LongTensor)
    train_dataset = Data.TensorDataset(train_X_toTensor, train_Y_toLongTensor)

    valid_X_toTensor = torch.Tensor(valid_X)
    valid_Y_toTensor = torch.Tensor(valid_Y)
    valid_Y_toLongTensor = valid_Y_toTensor.type(torch.LongTensor)
    valid_dataset = Data.TensorDataset(valid_X_toTensor, valid_Y_toLongTensor)

    test_X_toTensor = torch.Tensor(test_X)
    test_Y_toTensor = torch.Tensor(test_Y)
    test_Y_toLongTensor = test_Y_toTensor.type(torch.LongTensor)
    test_dataset = Data.TensorDataset(test_X_toTensor, test_Y_toLongTensor)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    dataloaders = {'train_dataset': train_loader, 'valid_dataset': valid_loader, 'test_dataset': test_loader}

    # ========================================================================================================================

    def initialize_vgg19(num_classes, feature_extract, pretrained=True):
        cnn_model = models.vgg19(pretrained=pretrained)
        if feature_extract:
            for parameter in cnn_model.parameters():
                parameter.set_parameter_requires_grad = False
        num_features = cnn_model.classifier[6].in_features
        cnn_model.classifier[6] = nn.Linear(num_features, num_classes)
        return cnn_model

    feature_extract = True
    cnn_model = initialize_vgg19(num_classes, feature_extract, pretrained=True)

    # ========================================================================================================================

    model_parameter = cnn_model.parameters()
    if feature_extract:
        model_parameter = []
        for name, parameter in cnn_model.named_parameters():
            if parameter.requires_grad == True:
                model_parameter.append(parameter)

    optimizer_ft = torch.optim.SGD(model_parameter, lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # ========================================================================================================================

    def train_model(model, criterion, optimizer, num_epochs=25):
        model.to(device)
        train_accuracy_history = []
        valid_accuracy_history = []
        train_losses = []
        valid_losses = []
        LRs = [optimizer.param_groups[0]['lr']]
        best_accuracy = 0
        best_model = copy.deepcopy(model.state_dict())
        since = time.time()

        for epoch in range(num_epochs):
            for phase in ['train_dataset', 'valid_dataset']:
                if phase == 'train_dataset':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train_dataset'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                        _, predictions = torch.max(outputs, 1)

                        if phase == 'train_dataset': 
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item()
                    running_corrects += torch.sum(predictions == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_accuracy = running_corrects.double() / len(dataloaders[phase].dataset)

                time_elapsed = time.time() - since
                print('Time elapsed in epoch {}: {:.0f}m {:.0f}s'.format(epoch + 1, time_elapsed // 60, time_elapsed % 60))
                print('{} Loss: {:.4f} Accuracy:{:.4f}'.format(phase, epoch_loss, epoch_accuracy))

                if phase == 'valid_dataset' and epoch_accuracy > best_accuracy:
                    best_accuracy = epoch_accuracy
                    best_model = copy.deepcopy(model.state_dict())

                if phase == 'valid_dataset':
                    valid_accuracy_history.append(epoch_accuracy)
                    valid_losses.append(epoch_loss)

                if phase == 'train_dataset':
                    train_accuracy_history.append(epoch_accuracy)
                    train_losses.append(epoch_loss)

            print('Optimizer learning rate: {:.7f}\n'.format(optimizer.param_groups[0]['lr']))
            LRs.append(optimizer.param_groups[0]['lr'])

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best valid accuracy: {:4f}\n'.format(best_accuracy))

        model.load_state_dict(best_model)
        return model, valid_accuracy_history, train_accuracy_history, valid_losses, train_losses, LRs

    # ========================================================================================================================

    cnn_model, valid_accuracy_history, train_accuracy_history, valid_losses, train_losses, LRs = train_model(cnn_model, criterion, optimizer_ft, num_epochs=num_epochs)

    # ========================================================================================================================

    # x1_axix = []
    # y1_axix = []
    # y2_axix = []

    # for i in range(num_epochs):
    #     x1_axix.append(i)
    #     y1_axix.append(round(train_accuracy_history[i].item() * 100, 2))
    #     y2_axix.append(round(valid_accuracy_history[i].item() * 100, 2))

    # fig, ax = plt.subplots()
    # plt.title("The accuracy in every epoch")
    # plt.xlabel("Epoch")
    # plt.ylabel("Accuracy (%)")
    # plt.plot(x1_axix, y1_axix, color="#00008B", marker='.', linestyle="-")
    # plt.plot(x1_axix, y2_axix, color="#006400", marker='.', linestyle="-")
    # for a, b in zip(x1_axix, y1_axix):
    #     plt.text(a, b, b, ha='center', va='bottom', fontsize=6)
    # for a, b in zip(x1_axix, y2_axix):
    #     plt.text(a, b, b, ha='center', va='bottom', fontsize=6)
    # plt.legend(['valid acc', 'test acc'])
    # plt.savefig("The accuracy of vgg19 model.png")

    x1_axix = []
    y1_axix = []
    y2_axix = []

    for i in range(num_epochs):
        x1_axix.append(i)
        y1_axix.append(round(train_accuracy_history[i].item() * 100, 2))
        y2_axix.append(round(valid_accuracy_history[i].item() * 100, 2))

    fig, ax = plt.subplots()
    plt.title("The accuracy in every epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.plot(x1_axix, y1_axix, color="#00008B", marker='.', linestyle="-")
    plt.plot(x1_axix, y2_axix, color="#006400", marker='.', linestyle="-")
    for a, b in zip(x1_axix, y1_axix):
        plt.text(a, b, b, ha='center', va='bottom', fontsize=6)
    for a, b in zip(x1_axix, y2_axix):
        plt.text(a, b, b, ha='center', va='bottom', fontsize=6)
    plt.legend(['valid acc', 'test acc'])
    plt.savefig("The accuracy of vgg19 model.png")

    # ========================================================================================================================

    def test_model(model):
        model.eval()
        running_corrects = 0
        for inputs, labels in dataloaders["test_dataset"]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            output = model(inputs)
            _, predictions = torch.max(output, 1)
            running_corrects += torch.sum(predictions == labels.data)
        prediction_accuracy = running_corrects.double() / len(dataloaders["test_dataset"].dataset)
        return prediction_accuracy.item() * 100

    print("The accuracy of the test dataset: ", test_model(cnn_model), "%")

    # ========================================================================================================================

In [ ]:
if __name__ == '__main__':
    Model_Vgg19()